In [1]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser
import pandas as pd
import num2words as n2w

In [2]:
# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')

def scrape(url):    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8'
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

In [ ]:
with open("urls.txt",'r') as urllist, open('data_reviews.csv','w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=["title","content","country","date","images","verified","author","rating","product","variant","url"],quoting=csv.QUOTE_ALL)
    writer.writeheader()
    plur=''
    for url_in in urllist.readlines():
        print('webpage:',url_in)
        url_part_1=url_in.split('&filterByStar=')[0].replace('_prev_','_next_')[:-1]
        print(url_part_1)
#        data = scrape(url_in+'&pageNumber='+str(page_n))
        for star in range(1,6):
            page_n=1
            while page_n<200:
                url=url_part_1+str(page_n)+'?filterByStar='+n2w.num2words(star)+'_star&pageNumber='+str(page_n)
                page_n+=1
                data = scrape(url)
            #    print(pd.DataFrame(data['reviews']).info())
                try:
                    for r in data['reviews']:
                #        print(r['rating'])
                        r["product"] = data["product_title"]
                        r['url'] = url
                        try:
                            if 'verified' in r:
                                if 'Verified Purchase' in r['verified']:
                #                    print(r['verified'])
                                    r['verified'] = 'Yes'
                                else:
                                    r['verified'] = 'No'
                        except TypeError:
                            r['verified'] = 'No'
                        try:
                            r['rating'] = r['rating'].split(' out of')[0]
                        except AttributeError:
            #                print('rating error')
            #                print(data)
                            r['rating'] = None
                        r['country']=((r['date'].split('on ')[0]).split('Reviewed in ')[1])[:-1].replace('the United States','USA')
                        date_posted = r['date'].split('on ')[-1]
                        if r['images']:
                            r['images'] = "\n".join(r['images'])
                        r['date'] = dateparser.parse(date_posted).strftime('%d %b %Y')
                        writer.writerow(r)
                        # sleep(5)
                except TypeError:
                    break

webpage: https://www.amazon.com/product-reviews/B07NVTGRVZ/ref=cm_cr_getr_d_paging_btm_prev_1&filterByStar=one_star&pageNumber=1
https://www.amazon.com/product-reviews/B07NVTGRVZ/ref=cm_cr_getr_d_paging_btm_next_
